In [91]:
%matplotlib inline

In [92]:
from __future__ import unicode_literals, print_function, division
import string
import random
import time
import os

import torch
import torch.nn as nn
from torch import optim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [93]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# load data

In [94]:
data_path = r'/home/alex/data/nlp/agmir/toy_data/trs_data'

In [95]:
from loaders import prepareReportData, normalizeString

In [96]:
input_lang, output_lang, ds = prepareReportData()

read data...
read 3580 reports
trimmed to 3580 sentence pairs
counting words...
counted words:
	 tags 590
	 report 1953


# more processing

In [97]:
from loaders import EOS_token, SOS_token

In [98]:
MAX_LENGTH = 30

In [99]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]

def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)[:MAX_LENGTH]

def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

In [100]:
report_pair = ds.__getitem__(100)[:2]
tensorsFromPair((report_pair[0], normalizeString(report_pair[1])))

(tensor([[2],
         [1]], device='cuda:0'),
 tensor([[  2],
         [  3],
         [  9],
         [ 10],
         [ 11],
         [169],
         [  3],
         [  3],
         [ 41],
         [ 94],
         [ 95],
         [ 25],
         [ 20],
         [ 21],
         [ 22],
         [175],
         [164],
         [  3],
         [ 41],
         [286],
         [ 25],
         [287],
         [  3],
         [  9],
         [ 27],
         [ 23],
         [ 29],
         [  3],
         [  9],
         [ 37]], device='cuda:0'))

# training routine

In [101]:
teacher_forcing_ratio = 0.5


def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [102]:
from utils import asMinutes, timeSince

In [103]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np

def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [104]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    
    training_pairs = []
    for i in range(n_iters):
        report_id = random.choice(range(len(ds)))
        report_pair = ds.__getitem__(report_id)[:2]
        report_pair = (report_pair[0], normalizeString(report_pair[1]))
        #print(report_pair)
        training_pairs.append(tensorsFromPair(report_pair))   
    
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

# eval routine

In [105]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

We can evaluate random sentences from the training set and print out the
input, target, and output to make some subjective quality judgements:




In [111]:
def evaluateRandomly(encoder, decoder, n=10):
    tuples = []
    
    for i in range(n):
        report_id = random.choice(range(len(ds)))
        report_pair = ds.__getitem__(report_id)[:2]
        pair = (report_pair[0], normalizeString(report_pair[1]))
        
        print('>', pair[0])
        print('=', pair[1])
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')
        tuples.append((pair[0], pair[1], output_sentence))
        
    return tuples

In [112]:
def evaluateAll(encoder, decoder):
    tuples = [] 
    
    for report_id in range(len(ds)):
        # get+process pair
        report_pair = ds.__getitem__(report_id)[:2]
        pair = (report_pair[0], normalizeString(report_pair[1]))
        
        # perform inference
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        tuples.append((pair[0], pair[1], output_sentence))
        
    return tuples

# train

In [116]:
from models import EncoderRNN, AttnDecoderRNN

In [117]:
%%time
hidden_size = 256
encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
attn_decoder1 = AttnDecoderRNN(hidden_size, output_lang.n_words, max_length=MAX_LENGTH, dropout_p=0.1).to(device)

trainIters(encoder1, attn_decoder1, 50000, print_every=1000)

1m 45s (- 85m 49s) (1000 2%) 4.3096
3m 19s (- 79m 57s) (2000 4%) 3.7561
4m 52s (- 76m 25s) (3000 6%) 3.6704
6m 20s (- 72m 50s) (4000 8%) 3.4716
7m 30s (- 67m 34s) (5000 10%) 3.5152
9m 3s (- 66m 23s) (6000 12%) 3.3609
10m 32s (- 64m 45s) (7000 14%) 3.2244
12m 1s (- 63m 6s) (8000 16%) 3.2900
13m 40s (- 62m 17s) (9000 18%) 3.2125
15m 13s (- 60m 52s) (10000 20%) 3.1200
16m 44s (- 59m 19s) (11000 22%) 3.2109
18m 15s (- 57m 49s) (12000 24%) 3.1770
19m 44s (- 56m 12s) (13000 26%) 3.1901
21m 20s (- 54m 53s) (14000 28%) 3.1033
22m 58s (- 53m 35s) (15000 30%) 3.0824
24m 30s (- 52m 5s) (16000 32%) 3.0624
26m 8s (- 50m 44s) (17000 34%) 3.0094
27m 42s (- 49m 16s) (18000 36%) 3.0398
29m 20s (- 47m 52s) (19000 38%) 2.9709
30m 53s (- 46m 19s) (20000 40%) 3.1022
32m 23s (- 44m 44s) (21000 42%) 2.9324
33m 59s (- 43m 15s) (22000 44%) 2.9677
35m 24s (- 41m 34s) (23000 46%) 2.9266
36m 52s (- 39m 56s) (24000 48%) 3.0072
38m 22s (- 38m 22s) (25000 50%) 2.9921
39m 49s (- 36m 45s) (26000 52%) 2.9128
41m 12s (-

# eval

In [118]:
from perf import get_av_bleu

In [119]:
pred_output = evaluateAll(encoder1, attn_decoder1)

In [120]:
get_av_bleu(pred_output)

0.08513800782339259